In [1]:
import os

import category_encoders as ce
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from IPython.display import display
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import GroupKFold, KFold, train_test_split
from sklearn.preprocessing import (KBinsDiscretizer, OrdinalEncoder,
                                   RobustScaler, normalize, MinMaxScaler)
from sklearn import model_selection


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
df = pd.read_csv("../data/train_5folds.csv")
test_df = pd.read_csv("../data/test.csv")

In [3]:


def add_features(df, transform="robust"):
    df["u_in_log1p"] = np.log1p(df["u_in"])
    df["u_in_power"] = np.power(df["u_in"], 2)
    
    df['time_delta'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta'] = df['time_delta'] * df['u_in']
    df['area'] = df.groupby('breath_id')['delta'].cumsum()

    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['one'] = 1
    df['count'] = (df['one']).groupby(df['breath_id']).cumsum()
    df['u_in_cummean'] =df['u_in_cumsum'] / df['count']
    
    df = df.drop(['count','one'], axis=1)

    for i in range(1, 8):    
        df[f'u_in_lag{i}'] = df.groupby('breath_id')['u_in'].shift(i)
        df[f'u_out_lag{i}'] = df.groupby('breath_id')['u_out'].shift(i)
        df[f'u_in_lag_back{i}'] = df.groupby('breath_id')['u_in'].shift(-i)
        df[f'u_out_lag_back{i}'] = df.groupby('breath_id')['u_out'].shift(-i)

    df = df.fillna(0)

    for i in range(1, 8):    
        df[f'u_in_diff{i}'] = df['u_in'] - df[f'u_in_lag{i}']

    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    

    r_map = {5: 0, 20: 1, 50: 2}
    c_map = {10: 0, 20: 1, 50: 2}
    rc_map = {'20_50': 0, '20_20': 1, '50_20': 2, '50_50': 3, '5_50': 4, '5_20': 5, '50_10': 6, '20_10': 7, '5_10': 8}
    df['RC'] = df['R'].astype(str) + '_' + df['C'].astype(str)
    
    df['R'] = df['R'].map(r_map)
    df['C'] = df['C'].map(c_map)
    
    df['RC'] = df['RC'].map(rc_map)
    return df


In [4]:
train_df = add_features(df)
test_df = add_features(test_df)

In [ ]:
not_used = ["id", "breath_id", "R", "C", "RC", "pressure", "kfold"]
used_columns = list(set(train_df.columns) - set(not_used))
norm_feature = sorted([c for c in sorted(used_columns) if "u_out" not in c])

In [11]:
# RS = RobustScaler()
# train_df[norm_feature] = RS.fit_transform(train_df[norm_feature])
# test_df[norm_feature] = RS.fit_transform(test_df[norm_feature])



In [12]:
train_df["u_in_round2"] = np.round(train_df["u_in"], 2)
test_df["u_in_round2"] = np.round(test_df["u_in"], 2)

In [13]:
train_df["kfold"] = -1
y = train_df["RC"].values

kf = model_selection.GroupKFold(n_splits=5)
# kf = model_selection.StratifiedGroupKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=train_df, y=y, groups=train_df.breath_id.values)):
    print(f)
    train_df.loc[v_, "kfold"] = f

0
1
2
3
4


In [14]:
train_df.to_csv("../data/train_5folds_nb13_robust.csv", index=False)
test_df.to_csv("../data/test_nb13_robust.csv", index=False)

In [15]:
sorted([c for c in sorted(used_columns)])

['area',
 'breath_id__u_in__diffmax',
 'breath_id__u_in__diffmean',
 'breath_id__u_in__max',
 'breath_id__u_out__max',
 'cross',
 'cross2',
 'delta',
 'time_delta',
 'time_step',
 'u_in',
 'u_in_cummean',
 'u_in_cumsum',
 'u_in_diff1',
 'u_in_diff2',
 'u_in_diff3',
 'u_in_diff4',
 'u_in_diff5',
 'u_in_diff6',
 'u_in_diff7',
 'u_in_diff8',
 'u_in_diff9',
 'u_in_lag1',
 'u_in_lag2',
 'u_in_lag3',
 'u_in_lag4',
 'u_in_lag5',
 'u_in_lag6',
 'u_in_lag7',
 'u_in_lag8',
 'u_in_lag9',
 'u_in_lag_back1',
 'u_in_lag_back2',
 'u_in_lag_back3',
 'u_in_lag_back4',
 'u_in_lag_back5',
 'u_in_lag_back6',
 'u_in_lag_back7',
 'u_in_lag_back8',
 'u_in_lag_back9',
 'u_in_log1p',
 'u_in_power',
 'u_out',
 'u_out_lag1',
 'u_out_lag2',
 'u_out_lag3',
 'u_out_lag4',
 'u_out_lag5',
 'u_out_lag6',
 'u_out_lag7',
 'u_out_lag8',
 'u_out_lag9',
 'u_out_lag_back1',
 'u_out_lag_back2',
 'u_out_lag_back3',
 'u_out_lag_back4',
 'u_out_lag_back5',
 'u_out_lag_back6',
 'u_out_lag_back7',
 'u_out_lag_back8',
 'u_out_lag_

In [2]:
train = pd.read_csv("../data/train_5folds_nb13_robust.csv")

In [3]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,kfold,u_in_log1p,u_in_power,time_delta,delta,area,cross,cross2,u_in_cumsum,u_in_cummean,u_in_lag1,u_out_lag1,u_in_lag_back1,u_out_lag_back1,u_in_lag2,u_out_lag2,u_in_lag_back2,u_out_lag_back2,u_in_lag3,u_out_lag3,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,u_in_lag5,u_out_lag5,u_in_lag_back5,u_out_lag_back5,u_in_lag6,u_out_lag6,u_in_lag_back6,u_out_lag_back6,u_in_lag7,u_out_lag7,u_in_lag_back7,u_out_lag_back7,u_in_diff1,u_in_diff2,u_in_diff3,u_in_diff4,u_in_diff5,u_in_diff6,u_in_diff7,breath_id__u_in__max,breath_id__u_out__max,breath_id__u_in__diffmax,breath_id__u_in__diffmean,RC,u_in_round2
0,1,1,1,2,-0.989052,-0.937384,0,5.837492,0,-1.100643,-0.779094,-15.475938,-0.860663,-0.734997,0.0,-0.665541,-0.725228,-0.648438,-0.864121,0.0,2.890207,0.0,-0.842527,0.0,3.664026,0.0,-0.819258,0.0,3.743915,0.0,-0.794152,0.0,4.277824,0.0,-0.76915,0.0,4.683263,0.0,-0.738446,0.0,4.686223,0.0,-0.703447,0.0,4.653368,0.0,0.471409,0.217993,0.121028,0.068315,0.023523,0.003111,-0.016488,0.112208,1,0.326941,1.273864,0,-0.94
1,2,1,1,2,-0.963608,3.049278,0,5.907794,0,0.878825,12.910990,0.103311,2.863270,-0.683536,0.0,-0.665541,-0.676829,0.162935,-0.847360,0.0,3.739712,0.0,-0.842527,0.0,3.724218,0.0,-0.819258,0.0,4.256189,0.0,-0.794152,0.0,4.661047,0.0,-0.76915,0.0,4.656595,0.0,-0.738446,0.0,4.621744,0.0,-0.703447,0.0,4.866757,0.0,113.467773,58.762670,38.091005,26.306250,18.058084,12.987738,10.056416,0.112208,1,-0.178436,-1.450205,0,3.05
2,3,1,1,2,-0.938006,3.948195,0,7.876254,0,1.011274,19.746583,0.200442,3.727568,-0.620131,0.0,-0.665541,-0.617568,0.555359,2.833140,0.0,3.801381,0.0,-0.825784,0.0,4.236036,0.0,-0.819258,0.0,4.639136,0.0,-0.794152,0.0,4.634403,0.0,-0.76915,0.0,4.592180,0.0,-0.738446,0.0,4.834887,0.0,-0.703447,0.0,4.957266,0.0,25.549576,71.696779,46.652513,32.222406,22.124535,15.915525,12.327665,0.112208,1,-0.292389,-2.064430,0,3.95
3,4,1,1,2,-0.912278,4.013452,0,11.742872,0,1.019963,20.296520,0.277152,3.811377,-0.555568,0.0,-0.665541,-0.557517,0.758212,3.663022,0.0,4.325760,0.0,2.850764,0.0,4.618643,0.0,-0.802501,0.0,4.612511,0.0,-0.794152,0.0,4.570046,0.0,-0.76915,0.0,4.805110,0.0,-0.738446,0.0,4.925291,0.0,-0.703447,0.0,4.665284,0.0,1.812565,14.110409,47.101127,32.651890,22.419739,16.128068,12.492546,0.112208,1,-0.300662,-2.109020,0,4.01
4,5,1,1,2,-0.886409,4.568332,0,12.234987,0,1.089713,25.266363,0.363024,4.361409,-0.483405,0.0,-0.665541,-0.490761,0.925096,3.723268,0.0,4.717757,0.0,3.679756,0.0,4.592042,0.0,2.877119,0.0,4.548200,0.0,-0.777379,0.0,4.782785,0.0,-0.76915,0.0,4.895425,0.0,-0.738446,0.0,4.633647,0.0,-0.703447,0.0,4.644297,0.0,15.753730,9.058184,14.415967,36.184309,24.929863,17.935321,13.894534,0.112208,1,-0.371002,-2.488167,0,4.57


In [4]:
train["u_out"].values

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
train["kfold"] = -1
y = train_df["RC"].values

kf = model_selection.GroupKFold(n_splits=10)
# kf = model_selection.StratifiedGroupKFold(n_splits=5)

for f, (t_, v_) in enumerate(kf.split(X=train_df, y=y, groups=train_df.breath_id.values)):
    print(f)
    train_df.loc[v_, "kfold"] = f